### **#1. Importing and Installing Dependencies**

In [ ]:
pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
from PIL import ImageFont, ImageDraw, Image 

### **#2. Keypoints using MP holistic**

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

### **#3. Keypoints using MP holistic**

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

### **#4. Setup Folders For Collection**

In [7]:
file = open("testwords.txt", encoding="utf8")
wordlist = []
for a in file:
    wordlist.append(a.strip())
print (wordlist)

['ጤና ይስጥልኝ', 'እግዚአብሔር', 'እናት', 'አባት', 'የኔ', 'አንድ', 'ሁለት', 'ሶስት', 'አራት']


In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data2') 

# Actions that we try to detect
actions = np.array(wordlist)

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [9]:
for action in actions:     
    for sequence in range(1, no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

### **#5. Collect Keypoint Values for Training and Testing**

In [10]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
   # Loop through actions
    for action in actions:        
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):         
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read() 

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                # Draw landmarks
                draw_styled_landmarks(image, results)                
                         
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    
                    pil_im = Image.fromarray(image)  

                    draw = ImageDraw.Draw(pil_im)  
                    # use a truetype font  
                    font = ImageFont.truetype("AbyssinicaSIL-Regular.ttf", 15)  

                    # Draw the text  
                    draw.text((15, 12), 'Collecting frame {} for {} Video Number {}'.format(frame_num, action, sequence), font=font) 
                    cv2image = np.array(pil_im)
#                     cv2_im_processed = cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)  
                    cv2.imshow('OpenCV Feed', cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGB))   
                    cv2.waitKey(500)
                else: 

                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    
                    pil_im = Image.fromarray(image)  

                    draw = ImageDraw.Draw(pil_im)  
                    # use a truetype font  
                    font = ImageFont.truetype("AbyssinicaSIL-Regular.ttf", 15)  

                    # Draw the text  
                    draw.text((15, 12), 'Collecting frame {} for {} Video Number {}'.format(frame_num, action, sequence), font=font) 
                    cv2image = np.array(pil_im)
#                     cv2_im_processed = cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)  
                    cv2.imshow('OpenCV Feed', cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGB))                 
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features


In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
label_map

{'ጤና ይስጥልኝ': 0,
 'እግዚአብሔር': 1,
 'እናት': 2,
 'አባት': 3,
 'የኔ': 4,
 'አንድ': 5,
 'ሁለት': 6,
 'ሶስት': 7,
 'አራት': 8}

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
np.array(sequences).shape

(270, 30, 1662)

In [16]:
np.array(labels).shape

(270,)

In [17]:
X = np.array(sequences)

In [18]:
X.shape

(270, 30, 1662)

In [19]:
X[4:5]

array([[[ 0.55280453,  0.45821354, -0.6513015 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55378425,  0.45731995, -0.55514681, ...,  0.47080112,
          0.62565899, -0.05532659],
        [ 0.55522048,  0.45701155, -0.5560258 , ...,  0.47293815,
          0.59770375, -0.01223224],
        ...,
        [ 0.55286497,  0.46165377, -0.7195521 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55218029,  0.462311  , -0.72336179, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55178207,  0.46241024, -0.7051295 , ...,  0.        ,
          0.        ,  0.        ]]])

In [20]:
y = to_categorical(labels).astype(int)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [22]:
print(y_test.shape)

(41, 9)


In [23]:
print(X_train.shape)

(229, 30, 1662)


In [24]:
print(y_train.shape)

(229, 9)


In [25]:
y_test.shape

(41, 9)

# 7. Build and Train LSTM Neural Network

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [27]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [28]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [29]:
model.compile(optimizer='AdaDelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [30]:
model.fit(X_train, y_train, epochs=2500, callbacks=[tb_callback])

Epoch 1/2500
8/8 [==============================] - 6s 119ms/step - loss: 2.1970 - categorical_accuracy: 0.1179
Epoch 2/2500
8/8 [==============================] - 1s 128ms/step - loss: 2.1965 - categorical_accuracy: 0.1179
Epoch 3/2500
8/8 [==============================] - 1s 130ms/step - loss: 2.1959 - categorical_accuracy: 0.1179
Epoch 4/2500
8/8 [==============================] - 1s 118ms/step - loss: 2.1954 - categorical_accuracy: 0.1179
Epoch 5/2500
8/8 [==============================] - 1s 130ms/step - loss: 2.1948 - categorical_accuracy: 0.1179
Epoch 6/2500
8/8 [==============================] - 1s 127ms/step - loss: 2.1943 - categorical_accuracy: 0.1179
Epoch 7/2500
8/8 [==============================] - 1s 122ms/step - loss: 2.1938 - categorical_accuracy: 0.1179
Epoch 8/2500
8/8 [==============================] - 1s 129ms/step - loss: 2.1933 - categorical_accuracy: 0.1179
Epoch 9/2500
8/8 [==============================] - 1s 127ms/step - loss: 2.1929 - categorical_accuracy:

8/8 [==============================] - 1s 139ms/step - loss: 2.1689 - categorical_accuracy: 0.2183
Epoch 74/2500
8/8 [==============================] - 1s 134ms/step - loss: 2.1685 - categorical_accuracy: 0.2183
Epoch 75/2500
8/8 [==============================] - 1s 136ms/step - loss: 2.1682 - categorical_accuracy: 0.2183
Epoch 76/2500
8/8 [==============================] - 1s 131ms/step - loss: 2.1679 - categorical_accuracy: 0.2183
Epoch 77/2500
8/8 [==============================] - 1s 133ms/step - loss: 2.1676 - categorical_accuracy: 0.2183
Epoch 78/2500
8/8 [==============================] - 1s 151ms/step - loss: 2.1672 - categorical_accuracy: 0.2183
Epoch 79/2500
8/8 [==============================] - 1s 136ms/step - loss: 2.1668 - categorical_accuracy: 0.2183
Epoch 80/2500
8/8 [==============================] - 1s 132ms/step - loss: 2.1665 - categorical_accuracy: 0.2183
Epoch 81/2500
8/8 [==============================] - 1s 138ms/step - loss: 2.1662 - categorical_accuracy: 0.21

8/8 [==============================] - 1s 130ms/step - loss: 2.1368 - categorical_accuracy: 0.3275
Epoch 146/2500
8/8 [==============================] - 1s 131ms/step - loss: 2.1360 - categorical_accuracy: 0.3275
Epoch 147/2500
8/8 [==============================] - 2s 213ms/step - loss: 2.1352 - categorical_accuracy: 0.3275
Epoch 148/2500
8/8 [==============================] - 1s 124ms/step - loss: 2.1343 - categorical_accuracy: 0.3275
Epoch 149/2500
8/8 [==============================] - 1s 126ms/step - loss: 2.1336 - categorical_accuracy: 0.3275
Epoch 150/2500
8/8 [==============================] - 1s 180ms/step - loss: 2.1326 - categorical_accuracy: 0.3275
Epoch 151/2500
8/8 [==============================] - 1s 148ms/step - loss: 2.1317 - categorical_accuracy: 0.3319
Epoch 152/2500
8/8 [==============================] - 1s 144ms/step - loss: 2.1308 - categorical_accuracy: 0.3319
Epoch 153/2500
8/8 [==============================] - 1s 136ms/step - loss: 2.1297 - categorical_accura

Epoch 217/2500
8/8 [==============================] - 1s 145ms/step - loss: 1.4935 - categorical_accuracy: 0.5284
Epoch 218/2500
8/8 [==============================] - 1s 140ms/step - loss: 1.4829 - categorical_accuracy: 0.5371
Epoch 219/2500
8/8 [==============================] - 1s 144ms/step - loss: 1.4830 - categorical_accuracy: 0.5371
Epoch 220/2500
8/8 [==============================] - 1s 138ms/step - loss: 1.4709 - categorical_accuracy: 0.5371
Epoch 221/2500
8/8 [==============================] - 1s 141ms/step - loss: 1.4555 - categorical_accuracy: 0.5284
Epoch 222/2500
8/8 [==============================] - 1s 150ms/step - loss: 1.4486 - categorical_accuracy: 0.5371
Epoch 223/2500
8/8 [==============================] - 1s 136ms/step - loss: 1.4407 - categorical_accuracy: 0.5371
Epoch 224/2500
8/8 [==============================] - 1s 147ms/step - loss: 1.4304 - categorical_accuracy: 0.5459
Epoch 225/2500
8/8 [==============================] - 1s 146ms/step - loss: 1.4204 - cat

8/8 [==============================] - 1s 120ms/step - loss: 0.9876 - categorical_accuracy: 0.6856
Epoch 289/2500
8/8 [==============================] - 1s 126ms/step - loss: 0.9749 - categorical_accuracy: 0.6943
Epoch 290/2500
8/8 [==============================] - 1s 124ms/step - loss: 0.9597 - categorical_accuracy: 0.7380
Epoch 291/2500
8/8 [==============================] - 1s 120ms/step - loss: 0.9583 - categorical_accuracy: 0.7424
Epoch 292/2500
8/8 [==============================] - 1s 129ms/step - loss: 0.9534 - categorical_accuracy: 0.7293
Epoch 293/2500
8/8 [==============================] - 1s 121ms/step - loss: 0.9546 - categorical_accuracy: 0.7380
Epoch 294/2500
8/8 [==============================] - 1s 124ms/step - loss: 0.9364 - categorical_accuracy: 0.7598
Epoch 295/2500
8/8 [==============================] - 1s 125ms/step - loss: 0.9278 - categorical_accuracy: 0.7642
Epoch 296/2500
8/8 [==============================] - 1s 119ms/step - loss: 0.9400 - categorical_accura

...

8/8 [==============================] - 1s 167ms/step - loss: 0.0536 - categorical_accuracy: 0.9825
Epoch 2211/2500
8/8 [==============================] - 1s 156ms/step - loss: 0.0503 - categorical_accuracy: 0.9869
Epoch 2212/2500
8/8 [==============================] - 1s 143ms/step - loss: 0.0478 - categorical_accuracy: 0.9913
Epoch 2213/2500
8/8 [==============================] - 1s 145ms/step - loss: 0.0464 - categorical_accuracy: 0.9869
Epoch 2214/2500
8/8 [==============================] - 1s 144ms/step - loss: 0.0485 - categorical_accuracy: 0.9869
Epoch 2215/2500
8/8 [==============================] - 1s 144ms/step - loss: 0.0479 - categorical_accuracy: 0.9913
Epoch 2216/2500
8/8 [==============================] - 1s 133ms/step - loss: 0.0477 - categorical_accuracy: 0.9825
Epoch 2217/2500
8/8 [==============================] - 1s 133ms/step - loss: 0.0479 - categorical_accuracy: 0.9913
Epoch 2218/2500
8/8 [==============================] - 1s 126ms/step - loss: 0.0472 - categorica

8/8 [==============================] - 1s 128ms/step - loss: 0.0483 - categorical_accuracy: 0.9825
Epoch 2282/2500
8/8 [==============================] - 1s 119ms/step - loss: 0.0490 - categorical_accuracy: 0.9869
Epoch 2283/2500
8/8 [==============================] - 1s 126ms/step - loss: 0.0444 - categorical_accuracy: 0.9869
Epoch 2284/2500
8/8 [==============================] - 1s 131ms/step - loss: 0.0601 - categorical_accuracy: 0.9825
Epoch 2285/2500
8/8 [==============================] - 1s 121ms/step - loss: 0.0406 - categorical_accuracy: 0.9869
Epoch 2286/2500
8/8 [==============================] - 1s 126ms/step - loss: 0.0625 - categorical_accuracy: 0.9825
Epoch 2287/2500
8/8 [==============================] - 1s 128ms/step - loss: 0.0434 - categorical_accuracy: 0.9869
Epoch 2288/2500
8/8 [==============================] - 1s 121ms/step - loss: 0.0436 - categorical_accuracy: 0.9869
Epoch 2289/2500
8/8 [==============================] - 1s 128ms/step - loss: 0.0430 - categorica

8/8 [==============================] - 1s 119ms/step - loss: 0.0404 - categorical_accuracy: 0.9869
Epoch 2353/2500
8/8 [==============================] - 1s 127ms/step - loss: 0.0402 - categorical_accuracy: 0.9869
Epoch 2354/2500
8/8 [==============================] - 1s 128ms/step - loss: 0.0387 - categorical_accuracy: 0.9913
Epoch 2355/2500
8/8 [==============================] - 1s 122ms/step - loss: 0.0385 - categorical_accuracy: 0.9869
Epoch 2356/2500
8/8 [==============================] - 1s 125ms/step - loss: 0.0374 - categorical_accuracy: 0.9869
Epoch 2357/2500
8/8 [==============================] - 1s 128ms/step - loss: 0.0414 - categorical_accuracy: 0.9869
Epoch 2358/2500
8/8 [==============================] - 1s 118ms/step - loss: 0.0388 - categorical_accuracy: 0.9913
Epoch 2359/2500
8/8 [==============================] - 1s 128ms/step - loss: 0.0380 - categorical_accuracy: 0.9869
Epoch 2360/2500
8/8 [==============================] - 1s 125ms/step - loss: 0.0393 - categorica

8/8 [==============================] - 1s 136ms/step - loss: 0.0355 - categorical_accuracy: 0.9869
Epoch 2424/2500
8/8 [==============================] - 1s 149ms/step - loss: 0.0344 - categorical_accuracy: 0.9913
Epoch 2425/2500
8/8 [==============================] - 1s 120ms/step - loss: 0.0356 - categorical_accuracy: 0.9913
Epoch 2426/2500
8/8 [==============================] - 1s 128ms/step - loss: 0.0343 - categorical_accuracy: 0.9869
Epoch 2427/2500
8/8 [==============================] - 1s 125ms/step - loss: 0.0381 - categorical_accuracy: 0.9869
Epoch 2428/2500
8/8 [==============================] - 1s 117ms/step - loss: 0.0383 - categorical_accuracy: 0.9869
Epoch 2429/2500
8/8 [==============================] - 1s 126ms/step - loss: 0.0351 - categorical_accuracy: 0.9869
Epoch 2430/2500
8/8 [==============================] - 1s 123ms/step - loss: 0.0357 - categorical_accuracy: 0.9913
Epoch 2431/2500
8/8 [==============================] - 1s 121ms/step - loss: 0.0351 - categorica

8/8 [==============================] - 1s 141ms/step - loss: 0.0316 - categorical_accuracy: 0.9913
Epoch 2495/2500
8/8 [==============================] - 1s 133ms/step - loss: 0.0330 - categorical_accuracy: 0.9956
Epoch 2496/2500
8/8 [==============================] - 1s 131ms/step - loss: 0.0320 - categorical_accuracy: 0.9913
Epoch 2497/2500
8/8 [==============================] - 1s 134ms/step - loss: 0.0324 - categorical_accuracy: 0.9869
Epoch 2498/2500
8/8 [==============================] - 1s 135ms/step - loss: 0.0304 - categorical_accuracy: 0.9956
Epoch 2499/2500
8/8 [==============================] - 1s 134ms/step - loss: 0.0314 - categorical_accuracy: 0.9913
Epoch 2500/2500
8/8 [==============================] - 1s 131ms/step - loss: 0.0318 - categorical_accuracy: 0.9913


In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 9)                 297       
                                                                 
Total params: 596,873
Trainable params: 596,873
Non-trai

# 8. Make Predictions

In [32]:
res = model.predict(X_test)

In [33]:
print (res[0])

[1.93622522e-08 2.03002375e-04 3.80136480e-04 5.59294358e-06
 3.14247009e-05 9.42295730e-01 3.37605327e-02 1.17966942e-02
 1.15268035e-02]


In [34]:
actions[np.argmax(res[5])]

'እግዚአብሔር'

In [35]:
actions[np.argmax(y_test[5])]

'እግዚአብሔር'

# 9. Save Weights

In [36]:
model.save('righthandsignmodel.h5')

In [37]:
model.load_weights('righthandsignmodel.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [38]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [39]:
yhat = model.predict(X_test)

In [40]:
print (yhat)

[[1.93622522e-08 2.03002375e-04 3.80136480e-04 5.59294358e-06
  3.14247009e-05 9.42295730e-01 3.37605327e-02 1.17966942e-02
  1.15268035e-02]
 [2.69673056e-15 1.52064416e-07 4.13091300e-04 3.82622517e-07
  2.03981872e-05 1.56815187e-03 9.96988595e-01 9.84087586e-04
  2.51296151e-05]
 [9.00144070e-10 9.99996305e-01 2.73695463e-07 1.40623729e-06
  2.79734067e-08 3.47039264e-09 7.06761513e-14 4.03617094e-07
  1.56907777e-06]
 [1.08153048e-12 7.19933189e-04 6.20166156e-06 9.98771250e-01
  3.23082386e-05 9.21212195e-05 2.78998527e-07 3.77433549e-04
  4.66529769e-07]
 [3.86384674e-10 9.99989510e-01 2.66924705e-09 2.27104442e-06
  6.21917593e-08 1.96559644e-11 1.19031583e-14 7.70087536e-06
  3.91893195e-07]
 [1.06162171e-11 9.99996424e-01 2.56521498e-10 3.28781357e-06
  1.38530326e-10 1.32795887e-13 5.12934491e-18 2.56847471e-07
  4.04974720e-09]
 [5.31414037e-18 1.34819661e-15 9.98956084e-01 1.84336000e-06
  7.93731713e-04 7.54079508e-07 2.46053998e-04 2.69963275e-14
  1.51666245e-06]
 [9.31

In [41]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [42]:
print(yhat)

[5, 6, 1, 3, 1, 1, 2, 5, 3, 6, 1, 7, 5, 3, 6, 7, 3, 5, 6, 3, 2, 0, 8, 1, 4, 7, 8, 8, 5, 5, 8, 4, 4, 6, 2, 3, 5, 8, 3, 5, 3]


In [43]:
print(ytrue)

[5, 6, 1, 3, 1, 1, 2, 5, 3, 5, 1, 7, 5, 3, 6, 7, 3, 5, 6, 3, 2, 0, 8, 1, 4, 7, 8, 8, 5, 5, 8, 4, 4, 5, 2, 3, 5, 8, 3, 5, 3]


In [44]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[40,  0],
        [ 0,  1]],

       [[36,  0],
        [ 0,  5]],

       [[38,  0],
        [ 0,  3]],

       [[33,  0],
        [ 0,  8]],

       [[38,  0],
        [ 0,  3]],

       [[31,  0],
        [ 2,  8]],

       [[36,  2],
        [ 0,  3]],

       [[38,  0],
        [ 0,  3]],

       [[36,  0],
        [ 0,  5]]], dtype=int64)

In [45]:
accuracy_score(ytrue, yhat)

0.9512195121951219

# 11. Test in Real Time

In [44]:
from scipy import stats

In [44]:
# 1. New detection variables
sequence = []
sentence = []
displaytext = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)  
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]          
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-20:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

#             with open('output.txt', 'a+') as f:
#                 f.write(sentence[-1] + ' ')
                
#             displaytext.append(sentence)
            print(sentence)
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         plt.imshow(image)
        pil_im = Image.fromarray(image)              

        draw = ImageDraw.Draw(pil_im)  
        # use a truetype font  
        font = ImageFont.truetype("AbyssinicaSIL-Regular.ttf", 25)  

        # Draw the text  
        draw.text((15, 12), '{}'.format(' '.join(sentence)), font=font) 
        
        cv2image = np.array(pil_im)

        cv2.imshow('OpenCV Feed', cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGB))          


        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

['ሶስት']
['ሶስት']
['ሶስት']
['ሶስት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ']

['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ', 'አንድ']
['ሶስት', 'ሁለት', 'አንድ', 'ደረጃ',

['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ',

['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ', 'ደረጃ']
['ደረጃ', 'አንድ', 'ደረጃ', 'አንድ',

['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
['ደረጃ', 'ሁለት', 'ደረጃ', 'መጣላት', 'ደረጃ']
[

KeyboardInterrupt: 

In [45]:
cap.release()
cv2.destroyAllWindows()